Import libraries

In [ ]:
import time
import os
import cv2 as cv
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard

TensorBoard setup for debugging

In [116]:
NAME = "{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
# in cmd type tensorboard --logdir=logs/

Grab dataset and prep for feeding to model

In [117]:
training_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = training_datagen.flow_from_directory('Dataset/Train', target_size=(225, 225), batch_size=32, class_mode='categorical')
test_set = training_datagen.flow_from_directory('Dataset/Test', target_size=(225, 225), batch_size=32, class_mode='categorical')

Found 416 images belonging to 2 classes.
Found 134 images belonging to 2 classes.


Model architecture, build, and train

In [118]:
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape=(225, 225, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(164, (3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(328, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(2, activation='softmax'))

# build and train
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(training_set, validation_data=test_set, epochs=15, steps_per_epoch=len(training_set), validation_steps=len(test_set), callbacks=[tensorboard])

Epoch 1/15
13/13 [==============================] - 2s 97ms/step - loss: 0.7085 - accuracy: 0.4952 - val_loss: 0.6780 - val_accuracy: 0.6791
Epoch 2/15
13/13 [==============================] - 1s 84ms/step - loss: 0.6918 - accuracy: 0.5288 - val_loss: 0.6675 - val_accuracy: 0.6791
Epoch 3/15
13/13 [==============================] - 1s 91ms/step - loss: 0.6787 - accuracy: 0.5505 - val_loss: 0.6484 - val_accuracy: 0.6642
Epoch 4/15
13/13 [==============================] - 1s 103ms/step - loss: 0.5506 - accuracy: 0.7260 - val_loss: 0.4564 - val_accuracy: 0.7612
Epoch 5/15
13/13 [==============================] - 1s 92ms/step - loss: 0.3690 - accuracy: 0.8534 - val_loss: 0.4337 - val_accuracy: 0.7612
Epoch 6/15
13/13 [==============================] - 1s 89ms/step - loss: 0.3788 - accuracy: 0.8317 - val_loss: 0.6367 - val_accuracy: 0.6940
Epoch 7/15
13/13 [==============================] - 1s 87ms/step - loss: 0.3197 - accuracy: 0.8558 - val_loss: 0.3664 - val_accuracy: 0.7985
Epoch 8/15
1

Summarize model

In [119]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_55 (Conv2D)          (None, 225, 225, 16)      448       
                                                                 
 max_pooling2d_55 (MaxPoolin  (None, 112, 112, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_56 (Conv2D)          (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_56 (MaxPoolin  (None, 56, 56, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_57 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 28, 28, 64)     

Apply model

In [120]:
DIR = os.getcwd() + r'\Dataset\Test'

wrong = 0
right = 0

cnt = 0

for i in os.listdir(DIR):
    path = os.path.join(DIR, i)
    for j in os.listdir(path):
        img = cv.imread(path+'/'+j)
        img = cv.resize(img, (225, 225))
        img = img.reshape(1, 225, 225, 3)
        img = img/255

        nodes = model.predict(img, verbose=0)
        a = np.argmax(nodes, axis=1)

        if a == 0:
            if i == 'Parasite': right += 1
            else: wrong += 1
            print(cnt, 'infected', nodes[0][0])
        else:
            if i == 'Uninfected': right += 1
            else: wrong += 1
            print(cnt, 'uninfected', nodes[0][1])
            
        cnt += 1

0 infected 1.0
1 infected 1.0
2 infected 1.0
3 infected 1.0
4 infected 1.0
5 infected 1.0
6 infected 1.0
7 infected 1.0
8 infected 0.9999819
9 uninfected 0.7822075
10 infected 1.0
11 infected 1.0
12 infected 0.9941215
13 uninfected 0.55659646
14 infected 1.0
15 infected 0.9997938
16 infected 0.77412593
17 infected 0.9999999
18 infected 1.0
19 infected 1.0
20 infected 1.0
21 infected 1.0
22 infected 0.95096016
23 infected 1.0
24 infected 1.0
25 infected 1.0
26 infected 0.9999999
27 infected 1.0
28 infected 1.0
29 infected 1.0
30 infected 1.0
31 infected 1.0
32 infected 1.0
33 infected 0.9939276
34 infected 0.9999341
35 infected 0.63415694
36 infected 0.9971413
37 infected 0.9630891
38 infected 0.7409377
39 infected 0.99999976
40 infected 0.99999666
41 infected 1.0
42 infected 1.0
43 infected 1.0
44 infected 1.0
45 infected 1.0
46 infected 0.8765917
47 infected 1.0
48 infected 1.0
49 infected 1.0
50 infected 1.0
51 infected 0.84301126
52 infected 1.0
53 infected 1.0
54 infected 0.9999996

Summary of model performance

In [121]:
total = len(os.listdir(os.getcwd() + r'\Dataset\Test\Uninfected')) + len(os.listdir(os.getcwd() + r'\Dataset\Test\Parasite'))

print(f'Correct: {(right/(right + wrong)) * 100}% --> {(right/(right + wrong)) * total}/{float(total)}')
print(f'Incorrect: {(wrong/(right + wrong)) * 100}% --> {(wrong/(right + wrong)) * total}/{float(total)}')

Correct: 94.77611940298507% --> 127.0/134.0
Incorrect: 5.223880597014925% --> 7.0/134.0


Save model

In [122]:
model.save('model.h5')